# Solving linear system of equations

The algorithm tries to solve the system of linear equations that can be represented as:

$$A\vec{x}=\vec{b}$$

The algorithm was proposed by [Aram W. Harrow, Avinatan Hassidim, and Seth Lloyd](https://arxiv.org/abs/0811.3171). It assumes that the eigenvalues ($\{\lambda_j,j=1,N\}$) and eigenstates ($\{u_j,j=1,N\}$) of matrix A of dimension $NxN$ are known. 

If $$\vec{b}=\sum_{j=1}^{M}\beta_j|u_j\rangle$$
and $C$ is a constant such as $C<\lambda_j$, then $\vec{x}$ can be approximated as

$$|x\rangle \approx C \sum_{j=1}^M \frac{\beta_j}{\lambda_j}|u_j\rangle$$

The algorithm uses three registers:

1. C or clock register
2. I to store $\vec{b}$
3. An ancilla qubit to get the solution

The steps of algorithm are:

1. Prepare an initial state $|\Psi_0\rangle^{c}\otimes|b\rangle^{I}$ with:$$|\Psi_0\rangle = \sqrt{\frac{2}{T}}\sum_{\tau=0}^{T-1}\sin\frac{\pi(\tau+\frac{1}{2})}{T}|\tau\rangle^C$$ and $$|b\rangle=\sum_{j=1}^{N-1}\beta_j|u_j>$$
2. Apply the conditonal Hamiltonian evolution $\sum_{\tau=0}^{T-1}|\tau\rangle\langle\tau|^C\otimes e^{iA\tau t_0/T}$, where $t_0=O(\kappa/\epsilon)$, being $\kappa$ the condition number of A (or the ratio between A’s largest and smallest eigenvalues) and $\epsilon$ the additive error achieved in the output state $|x\rangle$
3. Apply the QFT to register C, denoting the new basis sets as $|k\rangle$, for $k\in{0,...,T-1}$. Define $\hat{\lambda}:=2\pi k/t_0$ 
4. Append an ancilla register, S, and apply a controled rotation on S with C as the control. The rotation is defined so $|0\rangle$ is mapped for each $\lambda_j$ to $\sqrt{1-\frac{C^2}{\lambda_j^2}}|0\rangle+\frac{C}{\lambda_j}|1\rangle$
5. Uncompute the operations on C
6. Measure S. if result is $|1>$ then return the register I, else goto step 1.


However, the original HHL algorithm can be symplified as shown in [Cao et al.](https://arxiv.org/abs/1110.2232v2) and [Cao et al.](https://arxiv.org/abs/1110.2232v3). The state preparation of step 1 is changed to a Wals-Hadamard initialization of the Clock register and step 3 is calculated automatically, so the neither the eigenvectors or eigenvalues of the matrix is needed previously. But it needs ad additional quantum register to calculate the angles for the controled rotations (step 4) and two additional ancilla qubits. The general scheme is now (figure from [Cao et al.](https://arxiv.org/abs/1110.2232v3)):

<img src="Images/Cole.jpg"/>


Cole's algorithm steps are:

1. Prepare an initial state $(H|0\rangle)^{\otimes c}\otimes|b\rangle^{I}$
2. Make a Quanthum Phase Estimation of A on state $|b>$ and store it on register C
3. Calculate the rotations to apply from register C to register L using an unitary transformation $U_\lambda$
4. Apply a controled rotation on ancolla qubit S with L as the control.
5. Uncompute all the operations on ancillas and registers, except for ancilla qubit S
6. Measure S. if result is $|1>$ then return the register I, else goto step 1.

The system of linear equations to solve is

$\frac{1}{2}\begin{bmatrix}3&1\\1&3\end{bmatrix}\vec{x}=\begin{bmatrix}1\\0\end{bmatrix}$

The eigenvalues of this matrix are $\lambda_1=1$ and $\lambda_2=2$, with eigenvectors $|-\rangle=\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$ and $|+\rangle=\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$ 


In [1]:
from projectq.cengines import MainEngine
import numpy as np
import math

In [2]:
from projectq.ops import TimeEvolution,QubitOperator,_state_prep,get_inverse,Ry,Measure,QFT,All,H,X,Swap,Y,Z

In [3]:
from projectq.meta import Compute,Uncompute,Control,Dagger

These are helper functions to print the final state

In [4]:
def Display(string):
    from IPython.display import display, Markdown
    display(Markdown(string))

In [5]:
def get_state_as_str(eng,qubits,cheat=False):
    import numpy as np
    s="$"
    if (cheat):
        print("Cheat: ", eng.backend.cheat())
    if (len(qubits)==2):
        for i in range(4):
            #print("bits:%d%s"%(i,bits))
            a=eng.backend.get_amplitude("%d"%(i),qubits)
            if (abs(a.real)>0.0000001)|(abs(a.imag)>0.000001):
                if s!="$":
                    s=s+"+"
                a="({:.3f})".format(a)
                s=s+"%s|%d>"%(a,i)

    else:
        for j in range(2**(len(qubits)-1)):
            bits=np.binary_repr(j,width=len(qubits)-1)
            #print("Bits:",j,bits)
            for i in range(2):
                #print("bits:%d%s"%(i,bits))
                a=eng.backend.get_amplitude("%d%s"%(i,bits[-1::-1]),qubits)
                if (abs(a.real)>0.0000001)|(abs(a.imag)>0.000001):
                    if s!="$":
                        s=s+"+"
                    a="({:.3f})".format(a)
                    s=s+"%s|%s>_S|%s>_C|%d>_b"%(a,bits[0],bits[1:],i)
                #print(s)
    s=s+"$"
    #Display(s)
    return(s)

Helper funcion to solve the classical system and return the expectation values of Pauli's matrices

In [6]:
def solve(A,b):
    import numpy as np
    a = np.array(A)
    b = np.array(b)
    x = np.linalg.solve(a, b)
    sigmax=np.array([[0,1],[1,0]])
    sigmay=np.array([[0,-1j],[1j,0]])
    sigmaz=np.array([[1,0],[0,-1]])
    norm=np.linalg.norm(x)
    Esx=np.dot(x,np.dot(sigmax,x.T))/norm**2
    Esy=np.dot(x,np.dot(sigmay,x.T))/norm**2
    Esz=np.dot(x,np.dot(sigmaz,x.T))/norm**2
    return Esx,Esy,Esz,x

In [7]:
def DisplayResults(eng, Qureg,B,A,b):
    Display("After Measure:%s"%get_state_as_str(eng,Qureg))
    amplitude1=eng.backend.get_amplitude("0001",Qureg)
    amplitude2=eng.backend.get_amplitude("1001",Qureg)
    Esx,Esy,Esz,x = solve(A, b)
    amplitude1="({:.3f})".format(amplitude1)
    amplitude2="({:.3f})".format(amplitude2)
    Display("Result: (%s,%s). Classical: (%.3f,%.3f)"%(amplitude1,amplitude2,x[0],x[1]))
    Display("Calculated expectation value of $\sigma_X$:%.3f. Should be %.3f"%(eng.backend.get_expectation_value(QubitOperator("X0"),B).real,Esx))  
    Display("Calculated expectation value of $\sigma_Y$:%.3f. Should be %.3f"%(eng.backend.get_expectation_value(QubitOperator("Y0"),B).real,Esy)) 
    Display("Calculated expectation value of $\sigma_Z$:%.3f. Should be %.3f"%(eng.backend.get_expectation_value(QubitOperator("Z0"),B).real,Esz)) 

The matrix $A=\frac{1}{2}\begin{bmatrix}3&1\\1&3\end{bmatrix}$ can be decomposed on Pauli's matrices as:

$$ A=\frac{1}{2}(3I+\sigma_x)$$

So, to make the Hamiltonian time evolution (step 2), construct the Hamiltonian


In [8]:
Ham=QubitOperator("",1.5)
Ham=Ham+QubitOperator("X0",0.5)
print(Ham)

A=  np.array([[1.5,0.5], [0.5,1.5]])
print(A)

1.5 I +
0.5 X0
[[1.5 0.5]
 [0.5 1.5]]


The time step for the Hamiltonian time evolution in the Quantum Phase Estimation routine is selected to $t_0=2\pi$ and the total time is $T=4$

In [9]:
t0=2*math.pi
T=4

Now, design this circuit

<img src="Images/Cole2.png">

Initialize the Engine

In [10]:
eng=MainEngine()

Allocate the quantum registers. Three are needed:

1. A register to store the value of $\vec{b}$. In this case, only 1 qubit is needed.
2. A clock register, which needs only 2 qubits (one for each eigenvalue)
3. An ancilla qubit


In [11]:
B=eng.allocate_qureg(1)
C=eng.allocate_qureg(2)
S=eng.allocate_qureg(1)
Qureg=B+C+S

Define a Quantum Phase Estimation routine. 

In [12]:
def QPE(eng,C,b,Ham,t0,T):
        """
        Initiallise register C to Walsh-Hadamard state
        """
        All(H) |C
        """
        Apply the Hamiltonian time evolution. Take into account that ProjectQ adds a minus sign to the Time evolution (-i)
        """
        with Control(eng,C[0]):
            TimeEvolution(time=-1.*t0/T,hamiltonian=Ham) | b
        with Control(eng,C[1]):
            TimeEvolution(time=-2.*t0/T,hamiltonian=Ham) | b
        """
        And now, the inverse of the QFT 
        """
        Swap | (C[0],C[1])
        get_inverse(QFT)|C
        #QFT|C
        #Swap | (C[0],C[1])



Define the operator to calculate the rotations

In [13]:
def U_l(C):
        """
        The trick here is that U_l is only is swaping qubits on register C
        """
        Swap | (C[0],C[1])


Define the funtion to apply the controlled rotations on **S**.Parameter *r* must be calibrated, but Cole did it for r=4 to get good results

In [14]:
def ControlledRotations(eng,S,C,r):
    import math
    #Theta_1=math.pi
    #Theta_2=math.pi/3
    Theta_1=2*math.pi/(2**r)
    with Control(eng,C[1]):
        Ry(Theta_1) | S
    Theta_2=math.pi/(2**r)
    with Control(eng,C[0]):
        Ry(Theta_2) |S
    import math


Now, compose the full circuit

In [15]:
def prepare_state(state,b):
    from projectq.ops._state_prep import StatePreparation
    StatePreparation(state)|b

In [16]:
"""Prepare state b on register B
"""
b=[1,0]
prepare_state(b,B)
eng.flush()
Display(get_state_as_str(eng,Qureg))
with Compute(eng):
        """
        Apply QPE
        """
        QPE(eng,C,B,Ham,t0,T)
        #eng.flush()
        #Display("After QPE:%s"%get_state_as_str(eng,Qureg))


        """
        Apply U_l
        """
        U_l(C)
        #eng.flush()
        #Display("After Ul:%s"%get_state_as_str(eng,Qureg))
        
    
    


$(1.000+0.000j)|0>_S|00>_C|0>_b$

Now, apply the Controlled rotations. Parameter r must be calibrated, but Cole did it for r=4 to get good results

In [17]:
r=4
ControlledRotations(eng,S,C,r)
    
#eng.flush()
#print("After CR:",get_state_as_str(eng,Qureg))


In [18]:
Uncompute(eng)

eng.flush()
Display("After Uncompute:%s"%get_state_as_str(eng,Qureg))

After Uncompute:$(0.988+0.000j)|0>_S|00>_C|0>_b+(0.007-0.000j)|0>_S|00>_C|1>_b+(0.147+0.000j)|1>_S|00>_C|0>_b+(-0.049-0.000j)|1>_S|00>_C|1>_b$

Uncompute all the operations we did on C and B. projectQ will calculate automatically all the gates to apply

 
And measure. If the result for qubit S is 1, register B contains the solution


In [19]:
Measure | S
eng.flush()
Display("After Uncompute:%s"%get_state_as_str(eng,Qureg))
result=int(S)


After Uncompute:$(1.000+0.000j)|0>_S|00>_C|0>_b+(0.007-0.000j)|0>_S|00>_C|1>_b$

In [20]:
if result==0:
    All(Measure) | C
    Measure |B
    eng.flush()
    del eng
    print("Ohhh!!!! Bad luck! You have to reset and execute again. Or continue with the loop behind")

Ohhh!!!! Bad luck! You have to reset and execute again. Or continue with the loop behind


If the measurment of S is 1, register B has collapsed to the solution. Instead of multiples executions, using ProjectQ simulator it is possible to calculate the probabilitities (because the solution is real, they are the squared of the solution)

In [21]:
if (int(S)):
    DisplayResults(eng,Qureg,B,A,b)

If the previous exercise does not work well, execute the loop until the measurement of **S** is **1**

In [22]:
if result!=0:
    All(Measure) | C
    Measure | B
    eng.flush()
    del eng
result=0
while (result==0):
    eng=MainEngine()
    """
    Allocate the registers
    """
    B=eng.allocate_qureg(1)
    C=eng.allocate_qureg(2)
    S=eng.allocate_qureg(1)
    """Prepare state b on register B
    """
    Qureg=B+C+S
    
    prepare_state(b,B)
    with Compute(eng):
        """
        Apply QPE
        """
        QPE(eng,C,B,Ham,t0,T)
        """
        Apply U_l
        """
        U_l(C)
    eng.flush()
    r=4
    ControlledRotations(eng,S,C,r)
    eng.flush()
    Uncompute(eng)
    eng.flush()
    Measure | S
    eng.flush()
    result=int(S)
    if result==0:
        All(Measure) | C
        Measure |B
        eng.flush()
        del eng

In [23]:
if (int(S)):
    DisplayResults(eng,Qureg,B,A,b)    

After Measure:$(0.949+0.000j)|1>_S|00>_C|0>_b+(-0.314-0.000j)|1>_S|00>_C|1>_b$

Result: ((0.949+0.000j),(-0.314-0.000j)). Classical: (0.750,-0.250)

Calculated expectation value of $\sigma_X$:-0.597. Should be -0.600

/home/agomez/.conda/envs/cursoQC/lib/python3.5/site-packages/ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.


Calculated expectation value of $\sigma_Y$:-0.000. Should be 0.000

Calculated expectation value of $\sigma_Z$:0.802. Should be 0.800